## Preambule

In [167]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import pandas as pd
import json
import plotly

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [168]:
rules = ['PCC', 'ECPC', 'AP']
rulecolors = ['goldenrod', 'tomato', 'forestgreen']
year_of_focus = 2035

## Paths

In [169]:
path_main = Path("K:/Code/effort-sharing/")
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Paper_FairShares"

## Read data files

In [170]:
xr_dataread = xr.open_dataset(path_data / "startyear_2021/xr_dataread.nc")
xr_traj_yeari = xr.open_dataset(path_data / ("startyear_2021/Aggregated_files/xr_alloc_"+str(year_of_focus)+"_GHG_incl.nc"))
xr_alloc_yeari = xr.open_dataset(path_data / ("startyear_2021/Aggregated_files/xr_alloc_"+str(year_of_focus)+"_GHG_incl.nc"))
xr_total = xr.open_dataset(path_data / "xr_policyscen.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Get Cost-optimal output

In [171]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

## Compute fractions

World

In [172]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_World_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6['Region'] = 'WORLD'
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen = xr.Dataset.from_dataframe(dummy)
xr_scen = xr_scen.reindex(Time = np.arange(1850, 2101))
xr_scen = xr_scen.interpolate_na(dim="Time", method="linear")

Regional

In [173]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

R-10 regions

In [174]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_R10_regions_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r10 = xr.Dataset.from_dataframe(dummy)
xr_scen_r10 = xr_scen_r10.reindex(Time = np.arange(1850, 2101))
xr_scen_r10 = xr_scen_r10.interpolate_na(dim="Time", method="linear")

In [175]:
df_conv = pd.read_excel("X:/user/dekkerm/Data/AR6_regionclasses.xlsx", sheet_name = 'Sheet1')[['ISO', 'region_ar6_10']]
regs = np.array(df_conv.region_ar6_10)
regs[regs == 'Africa'] =            'R10AFRICA'
regs[regs == 'Eastern Asia'] =      'R10CHINA+'
regs[regs == 'Europe'] =            'R10EUROPE'
regs[regs == 'Southern Asia'] =     'R10INDIA+'
regs[regs == 'Latin America and Caribbean'] = 'R10LATIN_AM'
regs[regs == 'Middle East'] =       'R10MIDDLE_EAST'
regs[regs == 'North America'] =     'R10NORTH_AM'
regs[regs == 'Asia-Pacific Developed'] = 'R10PAC_OECD'
regs[regs == 'Eurasia'] =           'R10REF_ECON'
regs[regs == 'South-East Asia and developing Pacific'] = 'R10REST_ASIA'
df_conv['region_ar6_10'] = regs

In [176]:
native_regions = np.array(xr_scen_r.Region)
xrs = []
for cty in all_countries_iso:
    if cty not in native_regions:
        which_group = np.array(df_conv[df_conv.ISO == cty].region_ar6_10)[0]
        group_members = np.array(df_conv[df_conv.region_ar6_10 == which_group].ISO)
        emis_total = float(xr_dataread.GHG_hist.sel(Region=np.intersect1d(group_members,all_countries_iso), Time=2021).sum(dim='Region'))
        emis_frac = xr_dataread.GHG_hist.sel(Region=cty, Time=2021) / emis_total
        xrry = xr.merge([xr_scen_r10.sel(Region=which_group, Variable=['Emissions|Kyoto Gases'])*emis_frac,
                         xr_scen_r10.sel(Region=which_group, Variable=['Price|Carbon'])]).expand_dims({'Region': [cty]})
        xrs.append(xrry)
xr_scen_infilled_r = xr.concat(xrs, dim='Region')

Concatenate

In [177]:
xr_scen_all = xr.merge([xr_scen, xr_scen_r, xr_scen_infilled_r])

## Data for plots

In [178]:
settings_default = {'Temperature': 1.6,
                   'Risk': 0.5,
                   'NegEmis': 0.5,
                   'NonCO2red': 0.5,
                   'Timing': "Immediate",
                   'Convergence_year': 2050,
                   'Scenario': 'SSP2',
                   'Discount_factor': 0,
                   'Historical_startyear': 1990,}
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}
settings_ranges = {'Temperature': [1.6, 2.0],
                   'Risk': [0.5, 0.33],
                   'Convergence_year': [2050, 2080],
                    'NonCO2red':[0.33, 0.5, 0.67],
                   'Timing': ["Immediate"],
                   }
settings_ranges2 = {'Temperature': [1.6, 2.0],
                   'Risk': [0.5, 0.33],
                   'Convergence_year': [2050, 2080],
                    'NonCO2red':[0.33, 0.5, 0.67],
                   'Timing': ["Immediate"],
                   }
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}

In [179]:
ndcs = xr_dataread.GHG_ndc.mean(dim=['Conditionality', 'Ambition'])
data = xr_traj_yeari.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
data = xr_traj_yeari.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')

In [180]:
print("Preferences on default settings")
print('-----')
data = xr_traj_yeari.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')
argmax_default = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').fillna(-9e99).argmax(dim='variable2')

data = xr_traj_yeari.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
argmax_maximum = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').fillna(-9e99).argmax(dim=list(settings_default.keys())+['variable2'])
x = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').fillna(-9e99)
argmax_max = []
maxs = x.max(dim=list(settings_default.keys())+['variable2'])
for cty in x.Region:
    mx = float(maxs.sel(Region=cty).values)
    wh = np.where(x.sel(Region=cty).values == mx)
    wh = np.array(wh)[:, 0]
    argmax_max.append(wh[0])
argmax_max = np.array(argmax_max)

df_c = []
df_r_d = []
df_r_m = []
for cty_i, cty in enumerate(all_countries_iso):
    df_c.append(all_countries_names[cty_i])
    df_r_d.append(['ECPC', 'AP', 'PCC'][argmax_default.sel(Region=cty).values])
    df_r_m.append(['ECPC', 'AP', 'PCC'][argmax_max[cty_i]])
df_dict = {'Region': df_c, 'Default': df_r_d, 'Maximum': df_r_m}
df = pd.DataFrame(df_dict)
df.to_csv(path_data / "preferences_default_max.csv")

Preferences on default settings
-----


In [181]:
xr_policyscen = xr.open_dataset(path_data / "xr_policyscen.nc")
curpols = xr_policyscen.CurPol.sel(Time=year_of_focus).mean(dim='Model')
nzs = xr_policyscen.NetZero.sel(Time=year_of_focus).mean(dim='Model')

In [182]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [183]:
costopts = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').mean(dim='ModelScenario')

## Gaps

In [184]:
nec = xr_dataread.GHG_globe.sel(Time=year_of_focus, Temperature=1.6, Risk=0.5, NonCO2red=0.5, NegEmis=0.5, Timing='Immediate').values
gap_default = float(data_def.sel(Region=countries_iso).sum() - nec)
gap_default_perc = float((data_def.sel(Region=countries_iso).sum() - nec)/nec)

gap_max = float(data_max.sel(Region=countries_iso).sum() - nec)
gap_max_perc = float((data_max.sel(Region=countries_iso).sum() - nec)/nec)

## Gaps

In [185]:
q0 = 0.1
q1 = 0.9

In [186]:
co = costopts.Value
co_0 = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').quantile(q0, dim='ModelScenario')
co_1 = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').quantile(q1, dim='ModelScenario')
ndc = xr_dataread.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')
ndc = ndc.where(ndc.Region!='ZMB', np.nan)
ndc = ndc.where(ndc.Region!='COM', np.nan)
fair = xr_alloc_yeari.sel(Temperature=1.6,
                                Risk=0.5,
                                Timing='Immediate',
                                NonCO2red=0.5,
                                NegEmis=0.5,
                                Scenario='SSP2',
                                Convergence_year=2050,
                                Discount_factor=0,
                                Historical_startyear=1990)[['PCC', 'AP', 'ECPC']]
fair_0 = xr_alloc_yeari.sel(Temperature=1.6,
                            Risk=0.5,
                            Convergence_year=[2050, 2080])[['PCC', 'AP', 'ECPC']].quantile(0, dim=['Convergence_year', 'NegEmis', 'NonCO2red', 'Timing', 'Scenario', 'Discount_factor', 'Historical_startyear'])
fair_1 = xr_alloc_yeari.sel(Temperature=1.6,
                            Risk=0.5,
                            Convergence_year=[2050, 2080])[['PCC', 'AP', 'ECPC']].quantile(1, dim=['Convergence_year', 'NegEmis', 'NonCO2red', 'Timing', 'Scenario', 'Discount_factor', 'Historical_startyear'])

totaldev = fair - co # If positive, receive funds, if negative, pay funds (or partially domestic)
ndcdev = ndc - co # If positive, more domestic action needed, and vice versa
grey = (ndcdev - totaldev.where(totaldev > 0, 0)).where(ndcdev - totaldev.where(totaldev > 0, 0) > 0, 0)
red = -totaldev.where(totaldev < 0, 0)
green = totaldev.where(totaldev > 0, 0)
grey_super = (co-ndc).where(fair < co, 0)#-(ndcdev - totaldev.where(totaldev > 0, 0)).where(ndcdev - totaldev.where(totaldev > 0, 0) < 0, 0)
grey_super = grey_super.where(grey_super > 0, 0)

for cty in group_eu: # To avoid double counting!
    grey.loc[{'Region': cty}] = np.nan
    red.loc[{'Region': cty}] = np.nan
    green.loc[{'Region': cty}] = np.nan
    grey_super.loc[{'Region': cty}] = np.nan


k:\Environments\MyEnv\Lib\site-packages\numpy\lib\nanfunctions.py:1563: RuntimeWarning:

All-NaN slice encountered

k:\Environments\MyEnv\Lib\site-packages\numpy\lib\nanfunctions.py:1563: RuntimeWarning:

All-NaN slice encountered



## NDC 2035 data

In [187]:
df_ndc_raw = pd.read_excel("X:/user/dekkerm/Data/NDC/NDC tool GHG incl excl LULUCF with Input results.xlsx",
                            sheet_name='results_incl LULUCF', header=5)
regs = df_ndc_raw['country_name']
regs_iso = []
for r in regs:
    wh = np.where(countries_name == r)[0]
    if len(wh) == 0:
        if r == 'United States':
            regs_iso.append('USA')
        elif r == 'EU27':
            regs_iso.append('EU')
        elif r == 'Turkey':
            regs_iso.append('TUR')
        else:
            regs_iso.append(np.nan)
    else:
        regs_iso.append(countries_iso[wh[0]])
regs_iso = np.array(regs_iso)
df_ndc_raw['ISO'] = regs_iso

df_regs = []
df_amb = []
df_con = []
df_emis = []
df_lulucf = []
df_red = []
df_abs = []
df_inv = []
histemis = xr_dataread.GHG_hist.sel(Time=2015)
for r in list(countries_iso) + ['EU']:
    histemis_r = float(histemis.sel(Region=r))
    df_ndc_raw_sub = df_ndc_raw[df_ndc_raw['ISO'] == r]
    if len(df_ndc_raw_sub)>0:
        for cond in ['Unconditional', 'Conditional']:
            for ambition_i, ambition in enumerate(['min', 'max']):
                add = ['Min', 'Max'][ambition_i]
                key = 'NDC_2035_'+cond+'_'+add
                val = float(df_ndc_raw_sub[key])
                red = 1 - val / histemis_r
                abs_jones = histemis_r * (1 - red)
                df_regs.append(r)
                df_amb.append(ambition)
                df_con.append(cond)
                df_emis.append('NDC')
                df_lulucf.append('incl')
                df_red.append(red)
                df_abs.append(abs_jones)
                df_inv.append(val)

dict_ndc = {"Region": df_regs,
            "Ambition": df_amb,
            "Conditionality": df_con,
            "GHG_ndc_red": df_red,
            "GHG_ndc": df_abs,
            "GHG_ndc_inv": df_inv}
df_ndc = pd.DataFrame(dict_ndc)
xr_ndc = xr.Dataset.from_dataframe(df_ndc.set_index(["Region", "Ambition", "Conditionality"]))

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_11044\1402567247.py:38: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



In [198]:
costopt_mt = costopts.Value
ndc_est_mt = xr_dataread.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')
ndc_est_mt2035 = xr_ndc.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')
red_mt = xr_alloc_yeari.sel(Temperature=1.6,
                                 Risk=0.5,
                                 Timing='Immediate',
                                 NonCO2red=0.5,
                                 NegEmis=0.5,
                                 Scenario='SSP2',
                                 Convergence_year=2050,
                                 Discount_factor=0,
                                 Historical_startyear=1990)

emis_2015 = xr_dataread.GHG_hist.sel(Time=2015)
reductions = 1-red_mt/emis_2015
ndc_est = 1-(ndc_est_mt / emis_2015)
ndc_est2035 = 1-(ndc_est_mt2035 / emis_2015)
costopt = 1-costopt_mt/emis_2015

red_0 = 1-fair_0/emis_2015
red_1 = 1-fair_1/emis_2015
costopt_0 = 1-co_0/emis_2015
costopt_1 = 1-co_1/emis_2015

def data(reg): # Reduction targets
    return float(ndc_est.sel(Region=reg)), float(ndc_est2035.sel(Region=reg)), np.array(reductions.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()), float(costopt.sel(Region=reg))
def data_mt(reg): # Absolute emissions levels
    return float(ndc_est_mt.sel(Region=reg)), np.array(red_mt.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()), float(costopt_mt.sel(Region=reg))

## Finance plot

In [200]:
col1 = "#afc50f"#"rgba(255, 65, 54, 1)"
col2 = '#2e5061'#'rgba(61, 153, 112, 1)'
col3 = '#0062a6'

In [201]:
col_receive = 'forestgreen'
dx = 0.03
opac = 0.5
def areas(fig, ndc_estimate, reductions_i, costopt_i, row, col, yref=1):
    for i in range(3):
        ymin = yref + -wd*0.3 + [-wd*0.7, 0, wd*0.7][i]
        ymax = yref + wd*0.3+ [-wd*0.7, 0, wd*0.7][i]
        if reductions_i[i] > ndc_estimate:
            if costopt_i < reductions_i[i] and costopt_i > ndc_estimate:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=ndc_estimate+dx, x1=costopt_i-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    name='Increase domestic ambition',
                    showlegend=False,
                    opacity = opac,
                    legendgroup=3,
                    fillcolor='grey',  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt_i > reductions_i[i]:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=ndc_estimate+dx, x1=reductions_i[i]-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor='grey',  # Transparent fill color (adjust alpha for transparency)
                )
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=reductions_i[i]+dx, x1=costopt_i-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor=col_receive,  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt_i < ndc_estimate-0.0425:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=costopt_i+dx, x1=ndc_estimate-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor='rosybrown',  # Transparent fill color (adjust alpha for transparency)
                )
        # Pay abroad
        if reductions_i[i] > ndc_estimate:
            if costopt_i < ndc_estimate:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=ndc_estimate+dx, x1=reductions_i[i]-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    name='Pay for mitigation abroad',
                    showlegend=False,
                    opacity = opac,
                    legendgroup=3,
                    fillcolor='tomato',  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt_i < reductions_i[i]:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=costopt_i+dx, x1=reductions_i[i]-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor='tomato',  # Transparent fill color (adjust alpha for transparency)
                )
        # Receive from abroad
        if reductions_i[i] < ndc_estimate:
            fig.add_shape(
                row = row, col=col,
                type='rect',  # Draw a rectangle
                x0=reductions_i[i]+dx, x1=ndc_estimate-dx,  # The range of the box (between min and max values)
                y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                line=dict(color='black', width=0),  # Border color and width of the box
                name='Receive mitigation payment from abroad',
                showlegend=False,
                    opacity = opac,
                legendgroup=3,
                fillcolor=col_receive,  # Transparent fill color (adjust alpha for transparency)
            )
            fig.add_shape(
                row = row, col=col,
                type='rect',  # Draw a rectangle
                x0=ndc_estimate+dx, x1=costopt_i-dx,  # The range of the box (between min and max values)
                y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                line=dict(color='black', width=0),  # Border color and width of the box
                name='Receive mitigation payment from abroad',
                showlegend=False,
                opacity = opac,
                legendgroup=3,
                fillcolor=col_receive,  # Transparent fill color (adjust alpha for transparency)
            )

In [222]:
wd = 0.15

fig = make_subplots(rows=1, cols=1,
                    horizontal_spacing = 0., vertical_spacing=0.02)

fig.update_layout(height=1000, width=1000)
fs=20

# ============ #
# Actual data
# ============ #

fig.add_shape(
    col=1, row=1,
    type="line",
    x0=0, x1=0,
    y0=-1e3, y1=1e3,
    line=dict(color="black",  dash='dash',width=3)
)

REGS = ['USA', 'EU', 'JPN', 'BRA', 'IDN', 'VNM', 'CHN', 'PAK', 'IND', 'NGA']

ndcs = []
ndcs2 = []
reds = []
for reg_i, reg in enumerate(REGS):
    ndc_estimate, ndc2, reductions_i, costopt_i = data(reg)
    ndcs.append(ndc_estimate)
    ndcs2.append(ndc2)
    reds.append(reductions_i)
    cty = all_regions_names[all_regions_iso==reg][0].replace(' ', '<br>')
    if reg == 'USA': cty = 'United<br>States'
    fig.add_annotation(x=-0.05, y=reg_i*1., align = 'left',
                       text=cty, xref='paper', yref='y1', showarrow=False, font=dict(color='black', size=20))

    # WHISKERS
    c0 = -float(costopt_0.sel(Region=reg).Value)
    c1 = -float(costopt_1.sel(Region=reg).Value)
    for c in [c0, c1]:
        fig.add_shape(type="line", col=1, row=1, showlegend=False,
                    line=dict(color=col1, width=2),
                    x0=c, x1=c,
                    y0=-wd/2 + reg_i-wd , y1=wd/2 + reg_i-wd,
        )
    if reg_i ==0: sl = True
    else: sl = False
    fig.add_shape(type="line", col=1, row=1,
                line=dict(color=col1, width=4),
                showlegend=sl,
                name='Globally cost optimal',
                x0=c0, x1=c1,
                y0= reg_i-wd, y1= reg_i-wd,
    )

    # Horizontal lines between rules
    fig.add_trace(go.Scatter(x=[np.min(-reductions_i), np.max(-reductions_i)],
                             y=[reg_i+wd, reg_i+wd],showlegend=False,
                        line=dict(color=col3, width=2),
    ))

    # Horizontal lines between ndcs
    if not np.isnan(ndc2):
        fig.add_trace(go.Scatter(x=[-ndc_estimate, -ndc2],
                                y=[reg_i, reg_i],showlegend=False,
                            line=dict(color=col2, width=2),
        ))


# NDC 2030
fig.add_trace(go.Scatter(
    x=-np.array(ndcs),
    y=np.arange(len(REGS)),
    mode='markers',
    name='NDC 2030',
    showlegend=True,
    marker=dict(
        symbol='square',
        size=12,         # Marker size
        color='silver'     # Marker color
    )
))

# NDC 2035
fig.add_trace(go.Scatter(
    x=-np.array(ndcs2),
    y=np.arange(len(REGS)),
    mode='markers',
    name='NDC 2035',
    showlegend=True,
    marker=dict(
        symbol='square',
        size=12,         # Marker size
        color=col2     # Marker color
    )
))

# rules
for i in range(3):
    fig.add_trace(go.Scatter(
        x=-np.array(reds)[:, i],
        y=np.arange(len(REGS))+wd,
        mode='markers',
        name=['Per capita convergence', 'Ability to pay', 'Equal cumulative per capita'][i],
        showlegend=True,
        marker=dict(
            symbol=['circle', 'diamond', 'star'][i],
            size=12,         # Marker size
            color=col3     # Marker color
        )
    ))
fig.update_xaxes(row=1, col=1, range=(-1.5, 1.5), tickfont=dict(size=20), side='top',
                    tickvals=[-1.5, -1, -0.5, 0, 0.5,  1, 1.5],
                    ticktext=['150%', '100%', '(reduce)<br>50%',  '2015<br>emissions', '(increase)<br>50%', '100%', '150%'])

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
        legend=dict(
        title='',     # Optional title
        x=0.65, y=0.15,            # Positioning (top-right)
        bgcolor='rgba(255,255,255,0.5)',  # Semi-transparent background
        bordercolor='black',
        borderwidth=0,
        font=dict(size=20)
    ),
)
fig.update_yaxes(row=1, col=1, range=(-0.5, 10
                                      ), showticklabels=False)
fig.write_image("K:/Code/effort-sharing/Figures/COMMITTEDpolicybrief/Figure_"+str(year_of_focus)+".png", scale=5)
fig.show()